In [7]:
from hdmf.common import ExternalResources
from hdmf.common import DynamicTable
from hdmf import Data
import pandas as pd

er = ExternalResources(name='example')

In [8]:
key1 = er.add_key('key1')
key2 = er.add_key('key1')
er.add_ref(
            'uuid1', 'field1', key1, resource_name='resource1',
            resource_uri='resource_uri1', entity_id="id11", entity_uri='url11')
er.add_ref(
            'uuid2', 'field2', key2, resource_name='resource2',
            resource_uri='resource_uri2', entity_id="id12", entity_uri='url12')
er.add_ref(
            'uuid1', 'field1', er.get_key('key1', 'uuid1', 'field1'), resource_name='resource3',
            resource_uri='resource_uri3', entity_id="id13", entity_uri='url13')


(<hdmf.common.resources.Key at 0x7ff8e21ac0d0>,
 <hdmf.common.resources.Entity at 0x7ff8e21acf50>)

In [10]:
er.keys.data

[('key1',), ('key1',)]

In [27]:
er = ExternalResources('terms')
data = Data(name="species", data=['Homo sapiens', 'Mus musculus'])
er._check_object_field('uuid1','field1')
er._check_object_field(data,'field2')

In [29]:
data.object_id

'fed1e19c-2987-4c58-893f-c593bcb18e03'

In [23]:
er = ExternalResources('terms')
er.add_ref(
            container='uuid1', field='field1', key='key1', resource_name='resource1',
            resource_uri='resource_uri1', entity_id="id11", entity_uri='url11')
er.add_ref(
            container='uuid2', field='field2', key='key1', resource_name='resource2',
            resource_uri='resource_uri2', entity_id="id12", entity_uri='url21')

/Users/mavaylon/Research/hdmf_testing/hdmf/src/hdmf/container.py:181: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))


(<hdmf.common.resources.Key at 0x7f95869f7910>,
 <hdmf.common.resources.Entity at 0x7f95869b8b50>)

In [28]:
er.objects.data

[('uuid1', 'field1'), ('fed1e19c-2987-4c58-893f-c593bcb18e03', 'field2')]

In [18]:
er.add_ref(
            'uuid1', 'field1', 'key1', resource_name='resource1',
            resource_uri='resource_uri1', entity_id="id11", entity_uri='url11')
er.add_ref(
            'uuid2', 'field2', 'key1', resource_name='resource2',
            resource_uri='resource_uri2', entity_id="id12", entity_uri='url21')

er.add_ref(
            container='uuid1', field='field1', key=er.get_key('key1','uuid1', 'field1'),
            resource_name='resource1', resource_uri='resource_uri1', entity_id="id11", entity_uri='url11')
er.add_ref(
            container='uuid2', field='field2', key=er.get_key('key1','uuid2', 'field2'),
            resource_name='resource2', resource_uri='resource_uri2', entity_id="id12", entity_uri='url21')

(<hdmf.common.resources.Key at 0x7f95869f1c50>,
 <hdmf.common.resources.Entity at 0x7f95869f1a90>)

In [22]:
er.get_key('key1','uuid2', 'field2').idx

1

In [19]:
er.keys.to_dataframe()

,key
0,key1
1,key1


In [4]:
isinstance(er.get_key(key_name='key1'), Key)

NameError: name 'Key' is not defined

In [2]:
data = Data(name="species", data=['Homo sapiens', 'Mus musculus'])


In [3]:
er.add_ref(container=data, field='', key='Homo sapiens', resource_name='NCBI_Taxonomy',
           resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy', entity_id='NCBI:txid9606',
           entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=9606')



(<hdmf.common.resources.Key at 0x7ff8e144bcd0>,
 <hdmf.common.resources.Entity at 0x7ff8e20f6e50>)

In [9]:
er.get_keys()

,key_name,resources_idx,entity_id,entity_uri
0,Homo sapiens,0,NCBI:txid9606,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
1,Homo sapiens,0,NCBI:txid9606,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
2,Mus musculus,0,NCBI:txid10090,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...


In [6]:
data2 = Data(name="species2", data=['Homo sapiens', 'Mus musculus'])
data3 = Data(name="species3", data=['Homo sapiens', 'Mus musculus'])


In [7]:
er.add_ref(container=data2, field='f', key='Homo sapiens', resource_name='NCBI_Taxonomy',
           resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy', entity_id='NCBI:txid9606',
           entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=9606')

(<hdmf.common.resources.Key at 0x7f9bfb007a10>,
 <hdmf.common.resources.Entity at 0x7f9bfb007ad0>)

In [8]:
er.add_ref(container=data, field='', key='Mus musculus', resource_name='NCBI_Taxonomy',
           resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy', entity_id='NCBI:txid10090',
           entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=10090')

(<hdmf.common.resources.Key at 0x7f9bfb007510>,
 <hdmf.common.resources.Entity at 0x7f9bfb0071d0>)

In [19]:
key=er.get_key('Mus musculus',data,'')
er.keys.which(key='l')
from hdmf.container import Table, Row, Container, AbstractContainer

isinstance(data, AbstractContainer)

True

In [10]:
er.object_keys.to_dataframe()

,objects_idx,keys_idx
0,0,0
1,1,1
2,0,2


In [11]:
er.objects.to_dataframe()

,object_id,field
0,7a1246e7-5adb-4107-b3ab-19898bb81c40,
1,f27b6b3a-c99b-4404-9a75-8bf6c73857b7,f


In [12]:
er.keys.to_dataframe()

,key
0,Homo sapiens
1,Homo sapiens
2,Mus musculus


In [15]:
er.keys.which(key='Homo sapiens')

[0, 2]

In [27]:
object_field=er._check_object_field(data2,"f")
object_field.idx

1

In [26]:
er.object_keys['keys_idx', 2]

2

In [25]:
er.object_keys.which(objects_idx=object_field.idx)

[2]

In [ ]:
data2 = Data(name="species2", data=['Homo sapiens', 'Mus musculus'])
er.add_ref(container=data, field='', key=er.get_key2('Mus musculus',data,''), resource_name='NCBI_Taxonomy',
           resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy', entity_id='NCBI:txid10090',
           entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=10090')

In [ ]:
er.get_keys()

In [ ]:
er.object_keys.which(objects_idx=object_field.idx)

In [ ]:
er.object_keys[0]

In [ ]:
er.keys.which(key='Homo sapiens')

In [ ]:
er.get_key('Homo sapiens',data,'').idx

In [ ]:
er.object_keys['keys_idx', 1]

In [ ]:
er.keys.row[er.object_keys['keys_idx', object_field.idx]]

In [12]:
er.object_keys.to_dataframe()

,objects_idx,keys_idx
0,0,0
1,0,1


In [13]:
er.objects.to_dataframe()

,object_id,field
0,1e7187c7-a79e-44c8-9b75-77d26a229393,


In [14]:
er.keys.to_dataframe()

,key
0,Homo sapiens
1,Mus musculus


In [ ]:
er.get_key(key_name='Mus musculus')

In [ ]:
er.objects.which(object_id='data')

In [ ]:
er.objects.to_dataframe()

In [ ]:
objecttable_idx = er.objects.which(object_id=data.object_id)

In [ ]:
field_idx = er.objects.which(field='')
objecttable_idx = list(set(objecttable_idx) & set(field_idx))

In [ ]:
set(field_idx)

In [ ]:
objecttable_idx

In [ ]:
key_name='Mus musculus'
container=data2
field=''

In [ ]:
key_id = er.keys.which(key=key_name)
if container is not None and field is not None:
            # if same key is used multiple times, determine
            # which instance based on the Container
    object_field = er._check_object_field(container, field)
    key_tmp = er.object_keys['keys_idx', object_field.idx]
    print(key_tmp,key_id,object_field.idx)
    if key_tmp in key_id:
        print(er.keys.row[key_tmp],key_tmp,key_id)

In [ ]:
er.object_keys[object_field.idx]

In [ ]:
er.object_keys.to_dataframe()

In [ ]:
er.object_keys['keys_idx', object_field.idx]